# Практическая работа № 8


### 8.1. Искусственный нос

Цель – разработать и исследовать ИНС обратного распространения для искусственного носа, предназначенного для химического
анализа воздушной среды.

Задание

1. Исследовать и проанализировать имеющиеся экспериментальные данные (табл. 4.1), и определить количество вводов и
   выводов, требуемых для полносвязанной ИНС обратного распространения.
2. Создать и обучить нейронную сеть, которая будет способна указывать наличие определенных примесей в воздухе при
   анализе показаний химических датчиков.
3. Обучить нейронную сеть, расшив количество представительских выборок (обучающих пар), применяемых для обучения ИНС (
   табл. 4.2).
4. Определить оптимальную структуру нейронной сети с точки зрения минимизации среднеквадратической ошибки обучения.


In [ ]:
import numpy as np # noqa
import pandas as pd # noqa
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go


In [ ]:
class MLP(nn.Module):
    def __init__(self, input_amount=8, hidden_layers_amount=30, output_size=1):
        super().__init__()
        self.l1 = nn.Linear(input_amount, hidden_layers_amount)
        self.l2 = nn.Linear(hidden_layers_amount, output_size)

    def forward(self, x):
        x = torch.sigmoid(self.l1(x))
        x = torch.sigmoid(self.l2(x))
        return x

In [ ]:
X = torch.tensor(
    [
        [1, 0.05, 0.1, 0.3, 0.07, 0.08, 0.2, 0.05, 0.2, 0.6, 0.8],
        [0.8, 0.4, 0.7, 0.6, 0.1, 0.5, 1.0, 0.75, 0.5, 0.7, 0.8],
        [0.9, 0.2, 0.4, 0.5, 0.1, 0.7, 0.6, 0.5, 0.5, 0.7, 0.8],
        [0.85, 0.7, 0.8, 0.65, 0.1, 0.4, 1.0, 0.7, 0.4, 0.6, 0.7],
        [0.9, 0.3, 0.3, 0.4, 0.04, 0.1, 0.5, 0.3, 0.2, 0.7, 0.8],
        [0.95, 0.18, 0.21, 0.3, 0.05, 0.1, 0.3, 0.2, 0.2, 0.5, 0.7],
    ],
    dtype=torch.float32
)
y = torch.tensor(
    [
        [0, 0, 0, 0, 0, 1],  # Нет
        [1, 0, 0, 0, 0, 0],  # Ацетон
        [0, 1, 0, 0, 0, 0],  # Аммиак
        [0, 0, 1, 0, 0, 0],  # Изопропанол
        [0, 0, 0, 1, 0, 0],  # Белый «штрих»
        [0, 0, 0, 0, 1, 0],  # Уксус
    ],
    dtype=torch.float32
)
model = MLP(11, 10, 6)
mse = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
epochs = 3000
losses = []
for epoch in range(epochs):
    epoch_losses = []
    for x, y_val in zip(X, y):
        optimizer.zero_grad()
        outputs = model(x.view(1, -1))
        loss = mse(outputs, y_val)
        epoch_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    losses.append(np.mean(epoch_losses))
    rand_perm = torch.randperm(X.shape[0])
    X = X[rand_perm]
    y = y[rand_perm]

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=np.array(losses), mode='lines'))
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Loss',
    title='Training'
)
fig.show()

### 8.2. Прогнозирование

Цель – разработать и исследовать нейронную сеть обратного распространения, предназначенную для прогнозирования временных
серий, а также для анализа качества генератора случайных чисел.

Задание

1. Создать и обучить нейронную сеть, предназначенную для анализа временных серий заданной размерности и отражающую
   структуру данных серий. Окно скольжения равно 25, глубина прогноза: 2, число обучающих выборок: 8.
2. Осуществить прогноз значений будущих элементов временных серий.
3. Проверить работу НС и определить точность прогноза по формуле:
4. Создать и обучить две нейронные сети: предназначенную для анализа временных серий заданной размерности (окно
   скольжения равно 4, шаг скольжения S=1, период упреждения (шаг или глубина прогноза) m=1.) и для многофакторного
   прогнозирования (приложение 1).
5. Создать и обучить нейронную сеть для решения задачи многофакторного прогнозирования (исходные данные взять из
   практической работы № 2). Сравнить полученный результат по МГУА и НС.

#### 8.2.1 Создать и обучить нейронную сеть, предназначенную для анализа временных серий заданной размерности и отражающую структуру данных серий. Окно скольжения равно 25, глубина прогноза: 2, число обучающих выборок: 8.

In [73]:
def prepare_data(data, window_size, max_value):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    X = torch.stack(X) / max_value
    y = torch.stack(y).type(torch.int32)
    one_hot = torch.zeros((len(y), y.max() + 1))
    one_hot[torch.arange(len(y)), y] = 1
    y = one_hot
    return X, y

table = torch.tensor(
    [
        [0, 0, 8, 2, 2, 6, 3, 1, 3, 4],
        [0, 4, 0, 8, 0, 2, 9, 3, 7, 3],
        [6, 8, 7, 3, 1, 3, 4, 2, 8, 2],
        [4, 1, 8, 2, 7, 9, 4, 8, 8, 0],
        [1, 1, 5, 0, 5, 0, 7, 6, 7, 7],
        [5, 2, 6, 5, 9, 6, 9, 2, 6, 2],
        [0, 7, 4, 8, 5, 5, 9, 6, 3, 0],
        [7, 9, 7, 7, 1, 1, 9, 7, 5, 8],
        [6, 2, 0, 6, 2, 8, 1, 2, 5, 9],
        [1, 1, 2, 1, 5, 4, 2, 1, 6, 7]
    ],
    dtype=torch.float32
)
table = table.view(-1)
X, y = prepare_data(table, window_size=4, max_value=10)

In [74]:
class Conv1dNet(nn.Module):
    def __init__(self, input_amount, hidden_layers_amount, output_amount):
        super().__init__()
        _k_size = 4
        self.conv1d = nn.Conv1d(input_amount, hidden_layers_amount, kernel_size=_k_size)
        self.fc1 = nn.Linear(hidden_layers_amount, output_amount)

    def forward(self, x):
        x = self.conv1d(x)
        x = torch.relu(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x.squeeze()


model = Conv1dNet(1, 16, 10)
mse = nn.MSELoss()
optimizer = optim.Adam(model.parameters())


def train(X, y, num_epochs):
    losses = []
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output_tensor = model(X.unsqueeze(1))
        loss = mse(output_tensor, y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()

    return losses

In [75]:
epochs = 2000
losses = train(X, y, epochs)
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=np.array(losses), mode='lines'))
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Loss',
    title='Training'
)
fig.show()

future_X = torch.tensor([3, 1, 9, 4], dtype=torch.float32)
future_prediction = model(future_X.unsqueeze(0).unsqueeze(1))
print(f"Прогноз на основе значений {future_X}: {future_prediction.argmax()}")

Прогноз на основе значений tensor([3., 1., 9., 4.]): 7


#### 8.2.4. Создать и обучить две нейронные сети: предназначенную для анализа временных серий заданной размерности (окно скольжения равно 4, шаг скольжения S=1, период упреждения (шаг или глубина прогноза) m=1.) и для многофакторного прогнозирования (приложение 1).

In [76]:
table = torch.tensor(
    [
        [48, 14, 87, 28, 77, 97, 9, 14, 82, 2],
        [77, 3, 86, 20, 27, 67, 31, 12, 37, 42],
        [8, 47, 7, 84, 5, 29, 91, 36, 77, 32],
        [69, 84, 71, 30, 16, 32, 46, 24, 82, 27],
        [14, 14, 50, 2, 33, 0, 77, 65, 77, 70],
        [55, 20, 68, 59, 95, 64, 99, 24, 67, 29],
        [8, 77, 49, 88, 50, 57, 44, 68, 33, 0],
        [70, 98, 77, 74, 19, 14, 91, 78, 58, 86],
        [68, 28, 9, 62, 28, 87, 16, 27, 54, 96],
        [17, 15, 26, 17, 57, 49, 28, 15, 60, 52]
    ],
    dtype=torch.float32
)
table = table.view(-1)
X, y = prepare_data(table, window_size=4, max_value=100)

model = Conv1dNet(1, 16, 100)
mse = nn.MSELoss()
optimizer = optim.Adam(model.parameters())

future_X = torch.tensor([21, 32, 16, 33], dtype=torch.float32)
future_prediction = model(future_X.unsqueeze(0).unsqueeze(1))
print(f"Прогноз на основе значений {future_X}: {future_prediction.argmax()}")

Прогноз на основе значений tensor([21., 32., 16., 33.]): 3


#### 8.2.5 Создать и обучить нейронную сеть для решения задачи многофакторного прогнозирования (исходные данные взять из практической работы № 2). Сравнить полученный результат по МГУА и НС.

In [77]:
X = torch.tensor(
    [
        [0.986, 0.978, 0.970, 1.060, 0.880],
        [0.876, 0.858, 0.870, 1.082, 0.840],
        [0.699, 0.690, 0.764, 1.104, 0.480],
        [0.605, 0.619, 0.685, 1.126, 0.475],
        [0.514, 0.559, 0.566, 1.148, 0.525],
        [0.483, 0.492, 0.534, 1.170, 0.575],
        [0.459, 0.501, 0.570, 1.230, 0.582],
        [0.464, 0.506, 0.593, 1.304, 0.539],
        [0.478, 0.430, 0.640, 1.336, 0.512],
        [0.507, 0.587, 0.695, 1.370, 0.519],
        [0.671, 0.777, 0.730, 1.400, 0.617],
        [0.801, 1.109, 0.758, 1.430, 0.624],
        [0.981, 1.267, 0.794, 1.528, 0.634],
        [1.117, 1.425, 0.830, 1.626, 0.656],
        [1.254, 1.583, 0.866, 1.724, 0.682],
        [1.411, 1.741, 0.904, 1.824, 0.729],
        [1.568, 1.899, 1.075, 1.887, 0.780]
    ],
    dtype=torch.float32
)
y = torch.tensor(
    [
        [475.3],
        [413.5],
        [401.7],
        [400.9],
        [401.3],
        [402.5],
        [401.1],
        [404.4],
        [406.2],
        [412.2],
        [416.5],
        [425.8],
        [435.4],
        [445.4],
        [454.6],
        [466.1],
        [475.2]
    ],
    dtype=torch.float32
)

In [78]:
class LinRegNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = LinRegNet(input_size=5, hidden_size=10, output_size=1)
mse = nn.MSELoss()
optimizer = torch.optim.NAdam(model.parameters())

In [79]:
def train(X, y, num_epochs):
    losses = []
    for epoch in range(num_epochs):
        epoch_losses = []
        for x, y_val in zip(X, y):
            optimizer.zero_grad()
            outputs = model(x)
            loss = mse(outputs, y_val)
            epoch_losses.append(loss.item())
            loss.backward()
            optimizer.step()
        losses.append(np.mean(epoch_losses))
        rand_perm = torch.randperm(X.shape[0])
        X = X[rand_perm]
        y = y[rand_perm]
    return losses


epochs = 2000
losses = train(X, y, epochs)
print(f"Loss = {losses[-1]}:.5f")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=np.array(losses), mode='lines'))
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Loss',
    title='Training'
)
fig.show()

Loss = 126.52444301282658:.5f


In [80]:
with torch.inference_mode():
    new_data = torch.tensor([
        [0.864, 0.756, 0.918, 1.020, 0.940],
        [0.666, 0.889, 0.633, 1.056, 0.589]
    ], dtype=torch.float32)
    prediction = model(new_data)
    print(prediction)

tensor([[457.5833],
        [386.3224]])


#### Сравнение полученного результата по МГУА и НС

*_МГУА_*

In [81]:
data_x = np.array([
    [75.5, 25.2, 3343, 77],
    [78.5, 21.8, 3001, 78.2],
    [78.4, 25.7, 3101, 68],
    [77.7, 17.8, 3543, 77.2],
    [84.4, 15.9, 3237, 77.2],
    [75.9, 22.4, 3330, 77.2],
    [76, 20.6, 3808, 75.7],
    [67.5, 25.2, 2415, 62.6],
    [78.2, 20.7, 3295, 78],
    [78.1, 17.5, 3504, 78.2],
    [78.6, 19.7, 30565, 79],
    [84, 18.5, 3007, 67.6],
    [59.2, 54.4, 2844, 69.8],
    [90.2, 23, 2861, 68.4],
    [72.8, 20.2, 3259, 77.9],
    [67.7, 25.2, 3350, 78.1],
    [82.6, 22.4, 3344, 72.5],
    [74.4, 22.7, 2704, 66.6],
    [83.3, 18.1, 3642, 76.7],
    [83.7, 20.1, 2753, 68.8],
    [73.8, 17.3, 2916, 76.8],
    [79.2, 16.8, 3551, 78.1],
    [71.5, 29.9, 3177, 73.9],
    [75.3, 20.3, 3280, 78.6],
    [79, 14.1, 3160, 78.5]
])

data_y = np.array([0.904, 0.922, 0.763, 0.923, 0.918,
                   0.906, 0.905, 0.545, 0.894, 0.9,
                   0.932, 0.74, 0.701, 0.744, 0.921,
                   0.927, 0.802, 0.747, 0.927, 0.721,
                   0.913, 0.918, 0.833, 0.914, 0.923])

In [82]:
from gmdhpy.gmdh import MultilayerGMDH

Regressor = MultilayerGMDH
train_x, test_x = data_x[:20, :], data_x[20:, :]
train_y, test_y = data_y[:20], data_y[20:]
model = Regressor(max_layer_count=10)
model.fit(train_x, train_y)
predict_y = model.predict(test_x)

print(f"Средняя ошибка аппроксимации: {1 / len(test_y) * np.sum(abs((test_y - predict_y) / test_y))} %")
print(f"Коэффициент детерминации: {1 - np.sum((predict_y - test_y) ** 2) / np.sum((test_y - test_y.mean()) ** 2)}")
print(f"Средняя квадратичная ошибка: {np.mean((predict_y - test_y) ** 2)}")
print(f"Средняя абсолютная ошибка: {np.mean(np.abs(test_y - predict_y))}")

train layer0 in 0.00 sec
train layer1 in 0.02 sec
train layer2 in 0.00 sec
train layer3 in 0.02 sec
train layer4 in 0.01 sec
train layer5 in 0.00 sec
train layer6 in 0.02 sec
train layer7 in 0.02 sec
train layer8 in 0.00 sec
train layer9 in 0.02 sec
Средняя ошибка аппроксимации: 0.011619967272698865 %
Коэффициент детерминации: 0.8799299863028895
Средняя квадратичная ошибка: 0.00013704311083333423
Средняя абсолютная ошибка: 0.010424266249456138


*_НС_*

In [83]:
train_x = torch.tensor(train_x, dtype=torch.float32)
train_y = torch.tensor(train_y, dtype=torch.float32)
test_x = torch.tensor(test_x, dtype=torch.float32)

model = LinRegNet(4, 10, 1)
loss_fn = nn.MSELoss()
optimizer = optim.NAdam(model.parameters())
epochs = 3500
losses = train(train_x, train_y, epochs)
print(f"Loss: {losses[-1]}:.5f")

fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(epochs), y=np.array(losses), mode='lines'))
fig.update_layout(
    xaxis_title='Epochs',
    yaxis_title='Loss',
    title='Training'
)
fig.show()

with torch.inference_mode():
    y_pred = model(test_x)
    print(y_pred)

y_pred = y_pred.detach().numpy()
print(f"Средняя ошибка аппроксимации: {1 / len(test_y) * np.sum(abs((test_y - y_pred) / test_y))} %")
print(f"Коэффициент детерминации: {1 - np.sum((y_pred - test_y) ** 2) / np.sum((test_y - test_y.mean()) ** 2)}")
print(f"Средняя квадратичная ошибка: {np.mean((y_pred - test_y) ** 2)}")
print(f"Средняя абсолютная ошибка: {np.mean(np.abs(test_y - y_pred))}")

C:\Users\coolw\PycharmProjects\machine_learning\venv\Lib\site-packages\torch\nn\modules\loss.py:536: UserWarning:

Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.



Loss: 0.01132154298829846:.5f


tensor([[0.8374],
        [0.8374],
        [0.8374],
        [0.8374],
        [0.8374]])
Средняя ошибка аппроксимации: 0.35246835885983663 %
Коэффициент детерминации: -21.28289156862155
Средняя квадратичная ошибка: 0.005086560224152388
Средняя абсолютная ошибка: 0.06456649732589725
